001 データを読み込む

In [ ]:
import pandas as pd
customer_master = pd.read_csv('./input/01/customer_master.csv')
customer_master.head()

In [ ]:
customer_master.info()

顧客マスターのデータは5000件で全カラム`Nan`がないことが判明

`age`とか`registration_date`は数値や日付で読み込みたい

他のデータも読み込む

In [ ]:
item_master = pd.read_csv('./input/01/item_master.csv')
item_master.head()

In [ ]:
item_master.info()

商品マスターは、5種類で、こちらもデータに`NaN`はなし

取引データを読み込む  
`取引ID`と、`取引価格`、`取引日`、`顧客ID`が格納されている

In [ ]:
transaction_1 = pd.read_csv('./input/01/transaction_1.csv')
transaction_1.head()

In [ ]:
transaction_1.info()

In [ ]:
transaction_2 = pd.read_csv('./input/01/transaction_2.csv')
transaction_2.head()

In [ ]:
transaction_2.info()

取引の詳細データを読み込む  
`詳細ID`と`取引ID`、その取引の`商品ID`と`数量`が格納されている  
同じ`取引ID`で複数の`詳細ID`がある

In [ ]:
transaction_detail_1 = pd.read_csv('./input/01/transaction_detail_1.csv')
transaction_detail_1.head()

In [ ]:
transaction_detail_1.info()

In [ ]:
transaction_detail_2 = pd.read_csv('./input/01/transaction_detail_2.csv')
transaction_detail_2.head()

In [ ]:
transaction_detail_2.info()

データの読み込み完了

---
---

002 データを結合（ユニオン）する

In [ ]:
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True)
transaction.head()

In [ ]:
transaction.info()

5000件と1786件のデータが結合され、6786件の取引データになっている

In [ ]:
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True)
transaction_detail.head()

In [ ]:
transaction_detail.info()

In [ ]:
print(len(transaction_detail_1))
print(len(transaction_detail_2))

5000件と2144件のデータが結合され、7144件のデータになっている

---
---

003 売上データ同士の結合（ジョイン）

`transaction_id`を主軸に結合し`transaction_detail`に`payment_date` と`customer_id`を付加する。`price`は詳細の方の計算から求められる。

In [ ]:
join_transaction_detail = pd.merge(transaction_detail, transaction[['transaction_id', 'payment_date', 'customer_id']], on='transaction_id', how='left')
join_transaction_detail.head(100)

In [ ]:
join_transaction_detail.info()

7144件の詳細の件数と同じになっている

004 マスターデータ（顧客、製品）の結合（ジョイン）

`customer_id`をキーに、customer情報の全情報。 `item_id`をキーにitem情報を全情報結合する

In [ ]:
join_transaction_detail = pd.merge(join_transaction_detail, customer_master, on='customer_id', how='left')
join_transaction_detail = pd.merge(join_transaction_detail, item_master, on='item_id', how='left')
join_transaction_detail.head()

In [ ]:
join_transaction_detail.info()

7144件で結合がデータが欠損や増えたりはしていない  
データの結合完了  

---
---

005 必要なデータ列を追加する

売上の列が消えているので、ここでは詳細取引の売上列を追加する。  
`quantity`と`item_price`の掛け算で計算できる。

In [ ]:
join_transaction_detail['price'] = join_transaction_detail['item_price'] * join_transaction_detail['quantity']
join_transaction_detail[['quantity', 'item_price', 'price']].head()

データ加工完了

---
---

006 データの検算

結合したデータの売上合計と、元データ`transacation`の売上合計を確認して検算する。

In [ ]:
print(join_transaction_detail['price'].sum(), '円')
print(transaction['price'].sum(), '円')
join_transaction_detail['price'].sum() == transaction['price'].sum()

007 各種統計量を把握

欠損しているデータの数や全体の数値を把握する

In [ ]:
join_transaction_detail.isnull().sum()

欠損値なし!!

In [ ]:
join_transaction_detail.describe()

各列データの数量や平均、標準偏差、最小、最大、中央値(50%)、四分位数が把握できる。  
ただし、データの型が数値のものだけ。

In [ ]:
print(join_transaction_detail['payment_date'].min())
print(join_transaction_detail['payment_date'].max())


データの区間の確認

---
---

データの集積や分析

008 月別のデータを集計する

データの型を確認

In [ ]:
join_transaction_detail.dtypes

`payment_date`や`registration_date`を日付型に変換

In [ ]:
join_transaction_detail['payment_date'] = pd.to_datetime(join_transaction_detail['payment_date'])
join_transaction_detail['registration_date'] = pd.to_datetime(join_transaction_detail['registration_date'])
join_transaction_detail.info()

In [ ]:
join_transaction_detail['payment_month'] = join_transaction_detail['payment_date'].dt.strftime("%Y%m")
join_transaction_detail[['payment_date', 'payment_month']].head()

作成した`payment_month`を軸に売上を集計する

In [ ]:
join_transaction_detail.groupby('payment_month').sum()

In [ ]:
join_transaction_detail.groupby('payment_month').sum()['price']

009 商品別でデータを集計する

In [ ]:
join_transaction_detail.groupby('item_id').sum()['price']

商品別で月別の売上を集計・表示する

In [ ]:
join_transaction_detail.groupby(['payment_month', 'item_id']).sum()[['price','quantity']]

ピボットテーブルで表示する

In [ ]:
print(item_master)
pd.pivot_table(join_transaction_detail, index='item_name', columns='payment_month', values=['price', 'quantity'], aggfunc='sum')

010 商品別売上をグラフ💹で可視化する

売上をグラフで可視化

In [ ]:
graph_data = pd.pivot_table(join_transaction_detail, index='payment_month', columns='item_name', values='price', aggfunc='sum')
graph_data.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(list(graph_data.index), graph_data['PC-A'], label='PC-A')
plt.plot(list(graph_data.index), graph_data['PC-B'], label='PC-B')
plt.plot(list(graph_data.index), graph_data['PC-C'], label='PC-C')
plt.plot(list(graph_data.index), graph_data['PC-D'], label='PC-D')
plt.plot(list(graph_data.index), graph_data['PC-E'], label='PC-E')
plt.legend()

売上数量をグラフで可視化

In [ ]:
graph_data = pd.pivot_table(join_transaction_detail, index='payment_month', columns='item_name', values='quantity', aggfunc='sum')
graph_data.head()

In [ ]:
plt.plot(list(graph_data.index), graph_data['PC-A'], label='PC-A')
plt.plot(list(graph_data.index), graph_data['PC-B'], label='PC-B')
plt.plot(list(graph_data.index), graph_data['PC-C'], label='PC-C')
plt.plot(list(graph_data.index), graph_data['PC-D'], label='PC-D')
plt.plot(list(graph_data.index), graph_data['PC-E'], label='PC-E')
plt.legend()

---
---
---

`seaborn`を使ってグラフ化

In [ ]:
import seaborn as sns

In [ ]:
graph_sns_data =  pd.pivot_table(join_transaction_detail, index='payment_month', columns='item_name', values='quantity', aggfunc='sum')
graph_data.head()

In [ ]:
sns.lineplot(x='payment_month', y='PC-A', data=graph_sns_data)
sns.lineplot(x='payment_month', y='PC-B', data=graph_sns_data)
sns.lineplot(x='payment_month', y='PC-C', data=graph_sns_data)
sns.lineplot(x='payment_month', y='PC-D', data=graph_sns_data)
sns.lineplot(x='payment_month', y='PC-E', data=graph_sns_data)